In [13]:
#импортируем библиотеки
import joblib
import pandas as pd
import numpy as np
import lightgbm as lgb
from sklearn.model_selection import train_test_split
from sklearn.ensemble import VotingClassifier
from catboost import CatBoostClassifier
from sklearn.metrics import f1_score

In [18]:
# загружаем файлы
train = pd.read_csv('train_AIC.csv')
test = pd.read_csv('test_AIC.csv')

In [19]:
#отсортированные по важности признаки
film = ['is equal', 'Поставщик', 'Количество обработчиков',
       'Количество изменений после согласований', 'Месяц2', 'Группа закупок',
       'Количество позиций', 'Операционный менеджер', 'Согласование заказа',
       'День недели 2', 'Группа материалов', 'Месяц3', 'Сумма', 'Дней между',
       'Длительность', 'Материал', 'Месяц1', 'Изменение даты поставки 30',
       'Сумма * Количество', 'До поставки', 'Завод', 'Количество',
       'Месяц День недели 2', 'Категорийный менеджер', 'Вариант поставки',
       'Закупочная организация', 'Балансовая единица',
       'Изменение позиции заказа на закупку: изменение даты поставки на бумаге',
       'Количество циклов согласования', 'is equal_3',
       'Изменение позиции заказа на закупку: дата поставки', 'ЕИ',
       'Дней между_', 'is equal_2']

In [20]:
#преобразование тренировочного набора данных
train['НРП'] = np.where(train['НРП'] > 0, 1, 0)
train['ЕИ'] = np.where(train['ЕИ'] > 1, 1, 0)
train['Количество обработчиков'] = (train['Количество обработчиков 7'] + train['Количество обработчиков 15'] + train['Количество обработчиков 30'])/3
train['Дней между'] = (train['Дней между 0_1'] + train['Дней между 1_2'] + train['Дней между 2_3'] + train['Дней между 3_4'] + train['Дней между 4_5'] + train['Дней между 5_6']
                       + train['Дней между 6_7'] + train['Дней между 7_8'])/8
train['Согласование заказа'] = (train['Согласование заказа 1'] + train['Согласование заказа 2'] + train['Согласование заказа 3'])/3
train['Изменение позиции заказа на закупку: дата поставки'] = np.where(train['Изменение позиции заказа на закупку: дата поставки'] > 0, 1, 0)
train['Отмена полного деблокирования заказа на закупку'] = np.where(train['Отмена полного деблокирования заказа на закупку'] > 0, 1, 0)
train['Изменение позиции заказа на закупку: изменение даты поставки на бумаге'] = np.where(train['Изменение позиции заказа на закупку: изменение даты поставки на бумаге'] > 0, 1, 0)
train['is equal'] = train['До поставки'] == train['Длительность']
train['Месяц День недели 2'] = train['Месяц2'] + train['День недели 2']
train['Дней между_'] = (train['Дней между 0_1'] < 7) & (train['Дней между 1_2'] < 7) & (train['Дней между 2_3'] < 7) & (train['Дней между 3_4'] < 7)
train['is equal_2'] = ((train['Изменение позиции заказа на закупку: дата поставки'] == train['Отмена полного деблокирования заказа на закупку']) & 
                  (train['Изменение позиции заказа на закупку: дата поставки'] == train['Изменение позиции заказа на закупку: изменение даты поставки на бумаге']) &
                  (train['Отмена полного деблокирования заказа на закупку'] & train['Изменение позиции заказа на закупку: изменение даты поставки на бумаге']))
train['Сумма * Количество'] = train['Сумма'] * train['Количество']
train['is equal_3'] = ((train['Согласование заказа 1'] == train['Согласование заказа 2']) &  (train['Согласование заказа 1'] == train['Согласование заказа 3']) &
                       (train['Согласование заказа 2'] & train['Согласование заказа 3']))

In [21]:
#преобразуем тестовый датасет
test['НРП'] = np.where(test['НРП'] > 0, 1, 0)
test['ЕИ'] = np.where(test['ЕИ'] > 1, 1, 0)
test['Количество обработчиков'] = (test['Количество обработчиков 7'] + test['Количество обработчиков 15'] + test['Количество обработчиков 30'])/3
test['Дней между'] = (test['Дней между 0_1'] + test['Дней между 1_2'] + test['Дней между 2_3'] + test['Дней между 3_4'] + test['Дней между 4_5'] + test['Дней между 5_6']
                       + test['Дней между 6_7'] + test['Дней между 7_8'])/8
test['Согласование заказа'] = (test['Согласование заказа 1'] + test['Согласование заказа 2'] + test['Согласование заказа 3'])/3
test['Изменение позиции заказа на закупку: дата поставки'] = np.where(test['Изменение позиции заказа на закупку: дата поставки'] > 0, 1, 0)
test['Отмена полного деблокирования заказа на закупку'] = np.where(test['Отмена полного деблокирования заказа на закупку'] > 0, 1, 0)
test['Изменение позиции заказа на закупку: изменение даты поставки на бумаге'] = np.where(test['Изменение позиции заказа на закупку: изменение даты поставки на бумаге'] > 0, 1, 0)
test['is equal'] = test['До поставки'] == test['Длительность']
test['Месяц День недели 2'] = test['Месяц2'] + test['День недели 2']
test['Дней между_'] = (test['Дней между 0_1'] < 7) & (test['Дней между 1_2'] < 7) & (test['Дней между 2_3'] < 7) & (test['Дней между 3_4'] < 7)
test['is equal_2'] = ((test['Изменение позиции заказа на закупку: дата поставки'] == test['Отмена полного деблокирования заказа на закупку']) & 
                  (test['Изменение позиции заказа на закупку: дата поставки'] == test['Изменение позиции заказа на закупку: изменение даты поставки на бумаге']) &
                  (test['Отмена полного деблокирования заказа на закупку'] & test['Изменение позиции заказа на закупку: изменение даты поставки на бумаге']))
test['Сумма * Количество'] = test['Сумма'] * test['Количество']
test['is equal_3'] = ((test['Согласование заказа 1'] == test['Согласование заказа 2']) & (test['Согласование заказа 1'] == test['Согласование заказа 3']) &
                  (test['Согласование заказа 2'] == test['Согласование заказа 3']))
test = test[film[0:34]]

In [22]:
#разделение на тренировочную и валидационную выборку
new_x = train[film[0:34]]
new_y = train['y']

TEST_SIZE = 0.01
RANDOM_STATE = 42

train_X, test_X, train_y, test_y = train_test_split(new_x.values, new_y.values, test_size=TEST_SIZE, random_state=RANDOM_STATE)

In [7]:
#инициализация LGBMClassifier модели 
lgb_par = {'n_estimators': 3500, 'max_depth': 25, 'num_leaves': 247,
           'learning_rate': 0.07530361859735642, 'gamma': 5.757985966854834e-05,
           'boosting_type': 'dart', 'feature_fraction': 0.944735811995595,
           'bagging_fraction': 0.9772086347434312, 'bagging_freq': 7, 'min_child_samples': 1}

lgb_model = lgb.LGBMClassifier(**lgb_par, objective = 'binary')
lgb_model.fit(train_X, train_y)
f1_score(test_y, lgb_model.predict(test_X), average='binary')

In [8]:
#инициализация CatBoostClassifier модели
parameters = {'iterations': 1000, 'learning_rate': 0.0885430382096274,
                 'depth': 11, 'l2_leaf_reg': 8.5458685064206, 'bootstrap_type': 'Bayesian',
                 'random_strength': 1.5118792232990612e-07, 'bagging_temperature': 0.19937820660466068,
                 'od_type': 'IncToDec'}
index_features = [1, 15, 23, 7, 20, 25, 5, 26, 31, 10, 24, 9, 16, 4, 11, 30, 27, 29]

cat_model = CatBoostClassifier(**parameters, cat_features=index_features, verbose=250)
cat_model.fit(train_X, train_y)
f1_score(test_y, cat_model.predict(test_X), average='binary')

In [9]:
#инициализация VotingClassifier модели
voit_model = VotingClassifier(estimators=[('Cat', cat_model), ('lgb', lgb_model)], voting='soft')
voit_model.fit(train_X, train_y)
f1_score(test_y, voit_model.predict(test_X), average='binary')

In [11]:
#обучение на всём датасете
voit_model = VotingClassifier(estimators=[('Cat', cat_model), ('lgb', lgb_model)], voting='soft')
voit_model.fit(new_x.values, new_y.values)

0:	learn: 0.5581080	total: 889ms	remaining: 14m 47s
250:	learn: 0.1003395	total: 2m 45s	remaining: 8m 14s
500:	learn: 0.0753274	total: 5m 40s	remaining: 5m 39s
750:	learn: 0.0590218	total: 8m 40s	remaining: 2m 52s
999:	learn: 0.0466730	total: 11m 39s	remaining: 0us
[LightGBM] [Warning] Unknown parameter: gamma
[LightGBM] [Warning] feature_fraction is set=0.944735811995595, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.944735811995595
[LightGBM] [Warning] bagging_fraction is set=0.9772086347434312, subsample=1.0 will be ignored. Current value: bagging_fraction=0.9772086347434312
[LightGBM] [Warning] bagging_freq is set=7, subsample_freq=0 will be ignored. Current value: bagging_freq=7
[LightGBM] [Warning] Unknown parameter: gamma
[LightGBM] [Warning] feature_fraction is set=0.944735811995595, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.944735811995595
[LightGBM] [Warning] bagging_fraction is set=0.9772086347434312, subsample=1.0 will

VotingClassifier(estimators=[('Cat',
                              <catboost.core.CatBoostClassifier object at 0x0000021F58289BD0>),
                             ('lgb',
                              LGBMClassifier(bagging_fraction=0.9772086347434312,
                                             bagging_freq=7,
                                             boosting_type='dart',
                                             feature_fraction=0.944735811995595,
                                             gamma=5.757985966854834e-05,
                                             learning_rate=0.07530361859735642,
                                             max_depth=25, min_child_samples=1,
                                             n_estimators=3500, num_leaves=247,
                                             objective='binary'))],
                 voting='soft')

In [26]:
#сохранение прогноза модели
test['value'] = voit_model.predict(test)
test = test.rename_axis('id')
test['value'].sort_index().to_csv("Ai_Tach_submit.csv")

In [14]:
#сохранение модели в pkl файл
joblib.dump(voit_model, 'final_voiting.pkl')

['final_voiting.pkl']